# Demonstration of Algorithm on MNIST
---
## Import pacakges and load data

You will need to download the MNIST dataset locally to run this notebook

In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from model import make_tasks, ART_IPCA
from all_metrics import ART_IPCA_all_metrics

In [2]:
d_train = pd.read_csv('mnist_train.csv').values
d_test = pd.read_csv('mnist_test.csv').values

# Scale features in [0, 1]
X_train = d_train[:, 1:] / 255.0
X_test = d_test[:, 1:] / 255.0

y_train = d_train[:, 0]
y_test = d_test[:, 0]

# Shuffle data to be safe
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

del d_train, d_test

## ART_IPCA Model

Class interface in the style of scikit-learn with ```fit()``` and ```predict()``` functions

In [3]:
TX_train, Ty_train = make_tasks(X_train, y_train, n_tasks=5, n_spc=2000)

model = ART_IPCA(n_components=200, sim_metric=cosine_similarity, rho=0.5)
model.fit(TX_train, Ty_train)
y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

0.9413

## All Metrics

Code to create 'meta-tables' used to derive continual learning metrics (i.e. task-specific performance, overall performance, forgetting rate)

In [4]:
TX_test, Ty_test = make_tasks(X_test, y_test, n_tasks=5, n_spc=500)

mh_acc_mat, sh_acc_mat = ART_IPCA_all_metrics(TX_train, 
                                              Ty_train, 
                                              TX_test, 
                                              Ty_test,
                                              n_components=200,
                                              sim_metric=cosine_similarity, 
                                              rho=0.5)

Overall accuracy 
Multiheaded setting:  0.992
Singleheaded setting: 0.942


In [5]:
mh_acc_mat

array([[0.997,   nan,   nan,   nan,   nan],
       [0.997, 0.986,   nan,   nan,   nan],
       [0.997, 0.987, 0.997,   nan,   nan],
       [0.997, 0.987, 0.997, 0.999,   nan],
       [0.997, 0.986, 0.997, 0.999, 0.982]], dtype=float32)

In [6]:
sh_acc_mat

array([[0.997,   nan,   nan,   nan,   nan],
       [0.987, 0.982,   nan,   nan,   nan],
       [0.984, 0.966, 0.972,   nan,   nan],
       [0.976, 0.956, 0.955, 0.974,   nan],
       [0.969, 0.936, 0.903, 0.952, 0.952]], dtype=float32)